In [3]:
from pynq import Overlay
from pynq import MMIO

#def s32(value):
#    return -(value & 0x80000000) | (value & 0x7fffffff)




design = Overlay('./mnist_cnn.bit')



'''
print(design.ip_dict)
{'axi_cdma_0': {'phys_addr': 2116026368, 'addr_range': 65536, 'type': 'xilinx.com:ip:axi_cdma:4.1', 'state': None, 'interrupts': {}, 'gpio': {}, 'fullpath': 'axi_cdma_0', 'driver': <class 'pynq.overlay.DefaultIP'>}, 
'axi_gpio_0': {'phys_addr': 1092616192, 'addr_range': 65536, 'type': 'xilinx.com:ip:axi_gpio:2.0', 'state': None, 'interrupts': {}, 'gpio': {}, 'fullpath': 'axi_gpio_0', 'driver': <class 'pynq.lib.axigpio.AxiGPIO'>}}

'''
cdma_address = design.ip_dict['axi_cdma_0']['phys_addr']
axi_gpio_address = design.ip_dict['axi_gpio_0']['phys_addr']
 

zynq_addr = 0x30000000
cdma_ctrl = cdma_address+0x00
cdma_sa = cdma_address+0x18
cdma_da = cdma_address+0x20
cdma_btt = cdma_address+0x28

bram0_addr = 0xC0000000
bram1_addr = 0xC2000000
bram2_addr = 0xC4000000


zynq_sys = MMIO(zynq_addr,0x1500)
zynq_ans1 = MMIO(zynq_addr,0x4)
zynq_ans2 = MMIO(zynq_addr+4,0x4)
ctrl = MMIO(cdma_ctrl,0x18)
sa = MMIO(cdma_sa,0x1500)
da = MMIO(cdma_da,0x1500)
btt = MMIO(cdma_btt,0x10)






#mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
#mmio.write(ADDRESS_OFFSET, data)
#result = mmio.read(ADDRESS_OFFSET)

'''

initial input file to bram0
    
    
'''
print('start initial!\n')

bram0_offset = 0x0

with open("fix_input_0.txt", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram0_offset,int('0x'+line1,16))      
        bram0_offset += 4
 

'''


 write '1' into AXI_GPIO for start!
 
 
'''      
count=0


ctrl.write(0x0,0x04)
sa.write(0x0,zynq_addr)
da.write(0x0,bram0_addr)
btt.write(0x0,0x1500)


print('initial finish!\n')
gpio_a = MMIO(axi_gpio_address+8,8)
gpio_a.write(0x4,0)
gpio_a.write(0,1)

while count<100000:
    count+=1
    
gpio_a.write(0x4,0)
gpio_a.write(0,0)


'''
count=0
gpio_b = MMIO(axi_gpio_address,8)
gpio_b.write(0,0x0001)
gpio_b.write(0x4,0)
while count<100000:
    count+=1
gpio_b.write(0,0)
gpio_b.write(0x4,0)
'''

 
'''
 use 'a' to append
 use 'w' to truncate

'''

bram1_offset = 0x0

ctrl.write(0x0,0x04)
sa.write(0x0,bram1_addr)
da.write(0x0,zynq_addr)
btt.write(0x0,0x1500)

with open('output1.hex', 'w') as f_out:
    while bram1_offset<169*4:   # terminal condition may change to `if axi_gpio_io_o(finish) is false`
        ans = str(hex(zynq_sys.read(bram1_offset))).upper()[2:]
        if len(ans)!=8:
            ans = '0'*(8-len(ans))+ans
        f_out.write(ans+'\n')
        bram1_offset += 4

bram2_offset = 0x0

ctrl.write(0x0,0x04)
sa.write(0x0,bram2_addr)
da.write(0x0,zynq_addr)
btt.write(0x0,0x1500)

with open('output2.hex', 'w') as f_out:
    while bram2_offset<169*4:   # terminal condition may change to `if axi_gpio_io_o(finish) is false`
        ans = str(hex(zynq_sys.read(bram2_offset))).upper()[2:]
        if len(ans)!=8:
            ans = '0'*(8-len(ans))+ans
        f_out.write(ans+'\n')
        bram2_offset += 4
        
        
print('all finish!')



/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


start initial!

initial finish!

all finish!
